# Assignment 9: Convolutional Auto-Encoder

In this assignment, we show that it is possible to learn from unlabeled data using a convolutional auto-encoder network. 
The task is to reduce (a noisy version of) an image of the handwritten digits of MNIST into a deep feature representation, without making use of their labels, and reconstruct the sample from that representation.

For this purpose, we implement a convolutional auto-encoder and a denoising auto-encoder that learn a $K=10$-dimensional deep feature representation of each image, and uses this representation to reconstruct images to the original size of $28\times28$ pixels.
We show that such a network can be used to denoise images.

## Dataset

We will make use of the default implementation of the `torchvision.datasets.MNIST` dataset. 
MNIST has 10 labels of digit images.
However, besides the last task, we do not make use of the labels of the dataset, but we only utilize the images.
We instantiate the training and test sets of MNIST -- again we will use the test set for validation purposes.

### Task 1: Datasets

Instantiate the training and validation datasets of MNIST, i.e., split the training and validation sets, and make use of data loaders.
Select a simple `ToTensor` transform.
Instantiate a training data loader using a batch size of $B=32$, and a validation data loader with 100 samples in a batch.

In [ ]:
import torch
import torchvision

# training set and data loader
train_set = ...
train_loader = ...

# validation set and data loader
validation_set = ...
validation_loader = ...

### Task 2: Random Noise

Implement a function to generate noisy images by adding a uniform noise to the clean images:

$$\mathbf X_{\mathrm{noise}} = \mathbf X + \alpha \mathbf U^{D\times E}$$ 

where $\mathbf U$ is a tensor with entries generated from a uniform distribution between $-1$ and $1$. 
$D=E=28$ are the width and the height of the original image. 
To represent actual images, we will restrict the pixel values of $\mathbf X_{\mathrm{noise}}$ to be in the range $[0,1]$ by clipping any value that is outside that range.

Note that this function will also be used with batches of samples.

In [ ]:
def noise(x, alpha=0.5):
  # generate noise 
  N = ...
  # Add noise and clamp
  noisy_sample = ...

  return noisy_sample

### Noisy Image Visualization

Here we just assure that the images are correct, by displaying them to you.

In [ ]:
# get first validation image
input = validation_set[0][0]

# generate noise image
noisy = noise(input)

# plot images
from matplotlib import pyplot
pyplot.rcParams['image.cmap'] = 'gray'

fig, ax = pyplot.subplots(1, 2)
ax[0].imshow(input.squeeze())
ax[0].axis("off")
ax[1].imshow(noisy.squeeze())
ax[1].axis("off")
pyplot.show()

## Auto-Encoder Network

The auto-encoder network is composed of two parts: the encoder that transforms the input image to a deep feature representation; and the decoder that produces an image from such a deep feature.

For the encoder $\mathcal E$, we will basically use the same convolutional network topology as in the last assignment.
An exception is that we perform our down-sampling via striding and not via pooling.
After each convolution, we apply the ReLU activation.
The output of the encoder is a $K=10$ dimensional deep feature representation.
The complete encoder network topology can be found below in Topology 1(a).

The decoder $\mathcal D$ performs the inverse operations of the encoder.
A fully-connected layer is used to increase the number of samples to the same size as the output of the flattening of the encoder.
Then, a ReLU activation is applied. 
The flattening needs to be undone next by reshaping the vector into the correct dimensionality.
A fractionally-strided convolutional layer increases the intermediate representation by a factor of 2.
Note that the fractionally-strided convolution is implemented in `torch.nn.ConvTranspose2d`, and the `stride` parameter should have the same value as for the encoder.
Additionally, the `torch.nn.ConvTranspose2d` has a parameter `output_padding` which needs to be adapted to reach the correct output shape (see Test 1).
After this layer, we perform another ReLU activation and another fractionally-strided convolution to arrive at the original input dimension.
The complete decoder network topology can be found below in Topology 1(b).

Finally, we combine the two sub-networks into one auto-encoder network.
While there exist several possibilities for doing this, we will implement a third `torch.nn.Module` that contains an instance of the encoder and an instance of the decoder.

Topology 1: Network configurations of the (a) encoder and (b) decoder networks

(a) Encoder Network

*   2D convolutional layer with $Q_1$ channels, kernel size $5\times5$, **stride 2** and padding 2
*   activation function ReLU
*   2D convolutional layer with $Q_2$ channels, kernel size $5\times5$, **stride 2** and padding 2
*   flatten layer to convert the convolution output into a vector
*   activation function ReLU
*   fully-connected layer with the correct number of inputs and $K$ outputs

(b) Encoder Network

*   fully-connected layer with $K$ inputs and the correct number of outputs
*   activation function ReLU
*   reshaping to convert the vector into a convolution input
*   2D **fractionally-strided convolutional** layer with $Q_2$ channels, kernel size $5\times5$, stride 2 and padding 2
*   activation function ReLU
*   2D **fractionally-strided convolutional** layer with $Q_1$ channels, kernel size $5\times5$, stride 2 and padding 2

### Task 3: Encoder Network

Implement the encoder network for given parameters $Q_1$, $Q_2$, and $K$ as given in Topology 1(a).
Implement a network class that derives from `torch.nn.Module` and implement the `__init__` and the `forward` methods.

In [ ]:
class Encoder (torch.nn.Module):
  def __init__(self, Q1, Q2, K):
    # call base class constrcutor
    super(Encoder,self).__init__()
    # convolutional define layers
    self.conv1 = ...
    self.conv2 = ...
    # activation functions will be re-used for the different stages
    self.act = ...
    # define fully-connected layers
    self.flatten = ...
    self.fc = ...

  def forward(self, x):
    # get the deep feature representation
    deep_feature = ...
    return deep_feature

### Task 4: Decoder Network

Implement the decoder network for given parameters $Q_1$, $Q_2$, and $K$ as given in Topology 1(b).
Implement a network class that derives from `torch.nn.Module` and implement the `__init__` and the `forward` methods.
The output of the decoder network is supposed to have values in the range $[0,1]$, similar to the input values.
We need to make sure that only these values can be achieved.
Think of possible ways of doing that, and apply the way that seems most reasonable.

In [ ]:
class Decoder (torch.nn.Module):
  def __init__(self, Q1, Q2, K):
    # call base class constrcutor
    super(Decoder,self).__init__()
    # fully-connected layer
    self.fc = ...
    # convolutional layers
    self.deconv1 = ...
    self.deconv2 = ...
    # activation function
    self.act = ...

  def forward(self, x):
    # reconstruct the output image
    output = ...
    return output

### Task 5: Joint Auto-Encoder Network

Implement the auto-encoder network by combining the encoder and the decoder.
In the `__init__` function, instantiate an encoder from Task 3 and a decoder from Task 4.
In `forward`, pass the input through the encoder and the decoder: $\mathbf Y = \mathcal D(\mathcal E(\mathbf X))$

In [ ]:
class AutoEncoder(torch.nn.Module):
  def __init__(self, Q1, Q2, K):
    super(AutoEncoder,self).__init__()
    self.encoder = ...
    self.decoder = ...

  def forward(self,x):
    # encode input
    deep_feature = ...
    # decode to output
    reconstructed = ...
    return reconstructed

### Test 1: Output Sizes

The code below instantiates the auto-encoder network with $Q_1 = Q_2 = 32$ and $K=10$. 
Then the given input $\mathbf X$ is provided to the (untrained) auto-encoder network.
Use this code to verify that the deep feature extracted by the encoder and the output from the decoder part both have the desired size. 
Also, we verify that the output values are between 0 and 1.

If the tests cannot be passed, please check the implementations in Task 3, 4, and 5.

In [ ]:
# run on cuda device?
device = torch.device("cuda")
# create network
network = AutoEncoder(32, 32, 10).to(device)

# create or select a sample
x = torch.randn((1,1,28,28))

# use encoder to encode image and check its size
deep_features = network.encoder(x.to(device))
assert deep_features.shape[1] == 10

# use decoder to generate an image and check its size and value range
output = network.decoder(deep_features)
assert output.shape[2:] == (28,28)
assert torch.all(output >= 0) and torch.all(output <= 1)

## Training and Evaluation
We will implement two training procedures, a simple auto-encoder network and a denoising auto-encoder network, which should be combined into one function.
To train the network, we will use the $L_2$ distance between the output and the input of the network as a loss function, which is implemented in `torch.nn.MSELoss`:

  $$\mathcal J^{L_2} (\mathbf X, \mathbf Y) = \|\mathbf X - \mathbf Y\|^2$$

For optimization, we will make use of the `Adam` optimizer with a learning rate of $\eta=0.001$.
We will run the training for 10 epochs and compute training and validation set loss after each epoch.

Denoising training requires generating noisy images before forwarding them into the network and taking the loss between output and the clean image.

For evaluation, we will check whether some of the validation set samples are correctly reconstructed from the auto-encoder network and whether the noise is removed from the denoising network.



### Task 6: Training Loop

For a given network and learning rate, implement a function that initiates loss function (`torch.nn.MSELoss`), optimizer, and trains the network for 10 epochs on the training data.
If parameter `denoise` is `True`, generate noisy batch with factor `alpha`.
Compute the running average of the training loss for each epoch.
At the end of each epoch, also compute the validation set loss.

In [ ]:
def training_loop(network, lr=0.001, denoise=False, alpha=0.5):
  # define optimizer
  optimizer = ...  
  # define loss function
  loss = ...

  for epoch in range(10):
    # evaluate average loss for training and validation set
    train_loss = validation_loss = 0.

    for x,_ in train_loader:

      # generate noise images by factor alpha as input if denoise is True
      
      # compute network output
      y = ...

      # compute loss between output and input
      J = ...
      # perform update
      ...
      # accumulate loss
      train_loss += ...


    # compute validation loss
    with torch.no_grad():
      for x,t in validation_loader:
        # generate noise images by factor alpha as input if denoise is True

        # compute network output
        y = ...
        # compute loss
        J = ...
        # accumulate loss
        validation_loss += ...


    # print average loss for training and validation
    print(f"\rEpoch {epoch+1}; train: {train_loss/len(train_set):1.5f}, val: {validation_loss/len(validation_set):1.5f}")

### Task 7: Training of two Networks

Instantiate two `AutoEncoder` networks, one to train with only clean samples, and one to train with noisy images. Call the training loop for each network.

Note: If the training loss and validation loss do not decrease during training, try to reduce the learning rate (to $\eta=0.0005$ or even lower) and restart the training.
You will need to re-initialize the network, too.

In [ ]:
# define network
network1 = ...

# perform auto-encoder training 
training_loop(...)

In [ ]:
# define network
network2 = ...

# perform denoising auto-encoder training 
training_loop(...)

### Task 8: Reconstruction Result

This task is to visualize the reconstructed images from their originals.
For this purpose, select one image for each label from the first batch of the validation set. 
Generate the noisy input by Task 2.

Forward the clean images through the trained auto-encoder network to extract their reconstructions. 
Forward the noisy inputs through the trained denoising network to remove noise.

To show the difference, plot the original sample, reconstructed sample, noisy sample, and denoised sample for one label in one column. 
Make a single plot with 4 rows and 10 columns. 
See the reference plot in the slides.

In [ ]:
# get first validation set batch
x,t = next(iter(validation_loader))

# select one image for each label
...

# If required, convert the list of select images into tensor through torch.stack
original = ...

# compute reconstructed samples
reconstructed = ...
# generate noisy samples
noisy = ...
# compute denoised samples
denoised = ...

samples = [original, reconstructed, noisy, denoised]

# plot images
from matplotlib import pyplot
pyplot.rcParams['image.cmap'] = 'gray'

fig, axes = pyplot.subplots(nrows=..., ncols=..., figsize=(15, 6))

for i in range(...):
  for j in range(...):
    axes[i][j].imshow(samples[i][j].squeeze())
    axes[i][j].axis("off")